### Build model:

### Reducing memory load:
- crop images
- reduce batch size
- reduce number of channels (have 4 available)
- reduce number of base filters
- reduce depth

### working configurations:
    
depth = 4  
filter_num = 16  
model = isensee2017_model(input_shape=(1, 144,144,144), n_base_filters=filter_num, depth=depth, dropout_rate=0.3,
                      n_segmentation_levels=3, n_labels=4, optimizer=Adam, initial_learning_rate=5e-4,
                      loss_function=weighted_dice_coefficient_loss, activation_name="sigmoid")

In [1]:
import nibabel as nib

In [2]:
from functools import partial

from keras.layers import Input, LeakyReLU, Add, UpSampling3D, Activation, SpatialDropout3D
from keras.engine import Model
from keras.optimizers import Adam

import numpy as np
from keras import backend as K
from keras.engine import Input, Model
from keras.layers import Conv3D, MaxPooling3D, UpSampling3D, Activation, BatchNormalization, PReLU, Deconvolution3D
from keras.optimizers import Adam

# K.set_image_dim_ordering('th')
# K.set_image_dim_ordering('tf')
K.set_image_data_format('channels_first')

Keras_backend = K.backend()
if K=='tensorflow':
    K.set_image_dim_ordering('tf')
else:
    K.set_image_dim_ordering('th')

try:
    from keras.engine import merge
except ImportError:
    from keras.layers.merge import concatenate
    
def dice_coefficient(y_true, y_pred, smooth=1.):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coefficient_loss(y_true, y_pred):
    return -dice_coefficient(y_true, y_pred)


def weighted_dice_coefficient(y_true, y_pred, axis=(-3, -2, -1), smooth=0.00001):
    """
    Weighted dice coefficient. Default axis assumes a "channels first" data structure
    :param smooth:
    :param y_true:
    :param y_pred:
    :param axis:
    :return:
    """
    return K.mean(2. * (K.sum(y_true * y_pred,
                              axis=axis) + smooth/2)/(K.sum(y_true,
                                                            axis=axis) + K.sum(y_pred,
                                                                               axis=axis) + smooth))


def weighted_dice_coefficient_loss(y_true, y_pred):
    return -weighted_dice_coefficient(y_true, y_pred)


def label_wise_dice_coefficient(y_true, y_pred, label_index):
    return dice_coefficient(y_true[:, label_index], y_pred[:, label_index])


def get_label_dice_coefficient_function(label_index):
    f = partial(label_wise_dice_coefficient, label_index=label_index)
    f.__setattr__('__name__', 'label_{0}_dice_coef'.format(label_index))
    return f


dice_coef = dice_coefficient
dice_coef_loss = dice_coefficient_loss




def unet_model_3d(input_shape, pool_size=(2, 2, 2), n_labels=1, initial_learning_rate=0.00001, deconvolution=False,
                  depth=4, n_base_filters=32, include_label_wise_dice_coefficients=False, metrics=dice_coefficient,
                  batch_normalization=False, activation_name="sigmoid"):
    """
    Builds the 3D UNet Keras model.f
    :param metrics: List metrics to be calculated during model training (default is dice coefficient).
    :param include_label_wise_dice_coefficients: If True and n_labels is greater than 1, model will report the dice
    coefficient for each label as metric.
    :param n_base_filters: The number of filters that the first layer in the convolution network will have. Following
    layers will contain a multiple of this number. Lowering this number will likely reduce the amount of memory required
    to train the model.
    :param depth: indicates the depth of the U-shape for the model. The greater the depth, the more max pooling
    layers will be added to the model. Lowering the depth may reduce the amount of memory required for training.
    :param input_shape: Shape of the input data (n_chanels, x_size, y_size, z_size). The x, y, and z sizes must be
    divisible by the pool size to the power of the depth of the UNet, that is pool_size^depth.
    :param pool_size: Pool size for the max pooling operations.
    :param n_labels: Number of binary labels that the model is learning.
    :param initial_learning_rate: Initial learning rate for the model. This will be decayed during training.
    :param deconvolution: If set to True, will use transpose convolution(deconvolution) instead of up-sampling. This
    increases the amount memory required during training.
    :return: Untrained 3D UNet Model
    """
    inputs = Input(input_shape)
    current_layer = inputs
    levels = list()

    # add levels with max pooling
    for layer_depth in range(depth):
        layer1 = create_convolution_block(input_layer=current_layer, n_filters=n_base_filters*(2**layer_depth),
                                          batch_normalization=batch_normalization)
        layer2 = create_convolution_block(input_layer=layer1, n_filters=n_base_filters*(2**layer_depth)*2,
                                          batch_normalization=batch_normalization)
        if layer_depth < depth - 1:
            current_layer = MaxPooling3D(pool_size=pool_size)(layer2)
            levels.append([layer1, layer2, current_layer])
        else:
            current_layer = layer2
            levels.append([layer1, layer2])

    # add levels with up-convolution or up-sampling
    for layer_depth in range(depth-2, -1, -1):
        up_convolution = get_up_convolution(pool_size=pool_size, deconvolution=deconvolution,
                                            n_filters=current_layer._keras_shape[1])(current_layer)
        concat = concatenate([up_convolution, levels[layer_depth][1]], axis=1)
        current_layer = create_convolution_block(n_filters=levels[layer_depth][1]._keras_shape[1],
                                                 input_layer=concat, batch_normalization=batch_normalization)
        current_layer = create_convolution_block(n_filters=levels[layer_depth][1]._keras_shape[1],
                                                 input_layer=current_layer,
                                                 batch_normalization=batch_normalization)

    final_convolution = Conv3D(n_labels, (1, 1, 1))(current_layer)
    act = Activation(activation_name)(final_convolution)
    model = Model(inputs=inputs, outputs=act)

    if not isinstance(metrics, list):
        metrics = [metrics]

    if include_label_wise_dice_coefficients and n_labels > 1:
        label_wise_dice_metrics = [get_label_dice_coefficient_function(index) for index in range(n_labels)]
        if metrics:
            metrics = metrics + label_wise_dice_metrics
        else:
            metrics = label_wise_dice_metrics

    model.compile(optimizer=Adam(lr=initial_learning_rate), loss=dice_coefficient_loss, metrics=metrics)
    return model


def create_convolution_block(input_layer, n_filters, batch_normalization=False, kernel=(3, 3, 3), activation=None,
                             padding='same', strides=(1, 1, 1), instance_normalization=False):
    """
    :param strides:
    :param input_layer:
    :param n_filters:
    :param batch_normalization:
    :param kernel:
    :param activation: Keras activation layer to use. (default is 'relu')
    :param padding:
    :return:
    """
    layer = Conv3D(n_filters, kernel, padding=padding, strides=strides)(input_layer)
    if batch_normalization:
        layer = BatchNormalization(axis=1)(layer)
    elif instance_normalization:
        try:
            from keras_contrib.layers.normalization import InstanceNormalization
        except ImportError:
            raise ImportError("Install keras_contrib in order to use instance normalization."
                              "\nTry: pip install git+https://www.github.com/farizrahman4u/keras-contrib.git")
        layer = InstanceNormalization(axis=1)(layer)
    if activation is None:
        return Activation('relu')(layer)
    else:
        return activation()(layer)


def compute_level_output_shape(n_filters, depth, pool_size, image_shape):
    """
    Each level has a particular output shape based on the number of filters used in that level and the depth or number 
    of max pooling operations that have been done on the data at that point.
    :param image_shape: shape of the 3d image.
    :param pool_size: the pool_size parameter used in the max pooling operation.
    :param n_filters: Number of filters used by the last node in a given level.
    :param depth: The number of levels down in the U-shaped model a given node is.
    :return: 5D vector of the shape of the output node 
    """
    output_image_shape = np.asarray(np.divide(image_shape, np.power(pool_size, depth)), dtype=np.int32).tolist()
    return tuple([None, n_filters] + output_image_shape)


def get_up_convolution(n_filters, pool_size, kernel_size=(2, 2, 2), strides=(2, 2, 2),
                       deconvolution=False):
    if deconvolution:
        return Deconvolution3D(filters=n_filters, kernel_size=kernel_size,
                               strides=strides)
    else:
        return UpSampling3D(size=pool_size)







create_convolution_block = partial(create_convolution_block, activation=LeakyReLU, instance_normalization=True)


def isensee2017_model(input_shape=(1,144,144,144), n_base_filters=16, depth=5, dropout_rate=0.3,
                      n_segmentation_levels=3, n_labels=4, optimizer=Adam, initial_learning_rate=5e-4,
                      loss_function=weighted_dice_coefficient_loss, activation_name="sigmoid"):
    """
    This function builds a model proposed by Isensee et al. for the BRATS 2017 competition:
    https://www.cbica.upenn.edu/sbia/Spyridon.Bakas/MICCAI_BraTS/MICCAI_BraTS_2017_proceedings_shortPapers.pdf
    This network is highly similar to the model proposed by Kayalibay et al. "CNN-based Segmentation of Medical
    Imaging Data", 2017: https://arxiv.org/pdf/1701.03056.pdf
    :param input_shape:
    :param n_base_filters:
    :param depth:
    :param dropout_rate:
    :param n_segmentation_levels:
    :param n_labels:
    :param optimizer:
    :param initial_learning_rate:
    :param loss_function:
    :param activation_name:
    :return:
    """
    inputs = Input(input_shape)

    current_layer = inputs
    level_output_layers = list()
    level_filters = list()
    for level_number in range(depth):
        n_level_filters = (2**level_number) * n_base_filters
        level_filters.append(n_level_filters)

        if current_layer is inputs:
            in_conv = create_convolution_block(current_layer, n_level_filters)
        else:
            in_conv = create_convolution_block(current_layer, n_level_filters, strides=(2, 2, 2))

        context_output_layer = create_context_module(in_conv, n_level_filters, dropout_rate=dropout_rate)

        summation_layer = Add()([in_conv, context_output_layer])
        level_output_layers.append(summation_layer)
        current_layer = summation_layer

    segmentation_layers = list()
    for level_number in range(depth - 2, -1, -1):
        up_sampling = create_up_sampling_module(current_layer, level_filters[level_number])
        concatenation_layer = concatenate([level_output_layers[level_number], up_sampling], axis=1)
        localization_output = create_localization_module(concatenation_layer, level_filters[level_number])
        current_layer = localization_output
        if level_number < n_segmentation_levels:
            segmentation_layers.insert(0, create_convolution_block(current_layer, n_filters=n_labels, kernel=(1, 1, 1)))

    output_layer = None
    for level_number in reversed(range(n_segmentation_levels)):
        segmentation_layer = segmentation_layers[level_number]
        if output_layer is None:
            output_layer = segmentation_layer
        else:
            output_layer = Add()([output_layer, segmentation_layer])

        if level_number > 0:
            output_layer = UpSampling3D(size=(2, 2, 2))(output_layer)

    activation_block = Activation(activation_name)(output_layer)

    model = Model(inputs=inputs, outputs=activation_block)
    model.compile(optimizer=optimizer(lr=initial_learning_rate), loss=loss_function)
    return model


def create_localization_module(input_layer, n_filters):
    convolution1 = create_convolution_block(input_layer, n_filters)
    convolution2 = create_convolution_block(convolution1, n_filters, kernel=(1, 1, 1))
    return convolution2


def create_up_sampling_module(input_layer, n_filters, size=(2, 2, 2)):
    up_sample = UpSampling3D(size=size)(input_layer)
    convolution = create_convolution_block(up_sample, n_filters)
    return convolution


def create_context_module(input_layer, n_level_filters, dropout_rate=0.3, data_format="channels_first"):
    convolution1 = create_convolution_block(input_layer=input_layer, n_filters=n_level_filters)
    dropout = SpatialDropout3D(rate=dropout_rate, data_format=data_format)(convolution1)
    convolution2 = create_convolution_block(input_layer=dropout, n_filters=n_level_filters)
    return convolution2

Using TensorFlow backend.


In [3]:
depth = 4
filter_num = 16
model = isensee2017_model(input_shape=(1, 144,144,144), n_base_filters=filter_num, depth=depth, dropout_rate=0.3,
                      n_segmentation_levels=3, n_labels=4, optimizer=Adam, initial_learning_rate=5e-4,
                      loss_function=weighted_dice_coefficient_loss, activation_name="sigmoid")

## Make the labels and test train dictionaries:

In [4]:
# from glob import glob
# paths = glob('/Users/etheredgej/Desktop/MICCAI_BraTS17_Data_Training/train/HGG/*/')
# print(paths)

import os
HGG_dir_list = next(os.walk('./HGG/'))[1]
print(len(HGG_dir_list))
LGG_dir_list = next(os.walk('./LGG/'))[1]
print(len(LGG_dir_list))

210
75


In [5]:
completelist = HGG_dir_list + LGG_dir_list
# print(completelist[0:4])
np.random.shuffle(completelist) # shuffles in place
# print(completelist[0:4])

partition={}
train_percentage=0.7
partition['train']=completelist[0:int(len(completelist)*train_percentage)]
partition['test']=completelist[int(len(completelist)*train_percentage):len(completelist)]

labels={}
# HGG=0
# LGG=1
for directory in HGG_dir_list:
    labels[directory]=0
for directory in LGG_dir_list:
    labels[directory]=1

In [6]:
import numpy as np
from nilearn.image.image import check_niimg
from nilearn.image.image import _crop_img_to as crop_img_to


def crop_img(img, rtol=1e-8, copy=True, return_slices=False):
    """Crops img as much as possible
    Will crop img, removing as many zero entries as possible
    without touching non-zero entries. Will leave one voxel of
    zero padding around the obtained non-zero area in order to
    avoid sampling issues later on.
    Parameters
    ----------
    img: Niimg-like object
        See http://nilearn.github.io/manipulating_images/input_output.html
        img to be cropped.
    rtol: float
        relative tolerance (with respect to maximal absolute
        value of the image), under which values are considered
        negligeable and thus croppable.
    copy: boolean
        Specifies whether cropped data is copied or not.
    return_slices: boolean
        If True, the slices that define the cropped image will be returned.
    Returns
    -------
    cropped_img: image
        Cropped version of the input image
    """

    img = check_niimg(img)
    data = img.get_data()
    infinity_norm = max(-data.min(), data.max())
    passes_threshold = np.logical_or(data < -rtol * infinity_norm,
                                     data > rtol * infinity_norm)

    if data.ndim == 4:
        passes_threshold = np.any(passes_threshold, axis=-1)
    coords = np.array(np.where(passes_threshold))
    start = coords.min(axis=1)
    end = coords.max(axis=1) + 1

    # pad with one voxel to avoid resampling problems
    start = np.maximum(start - 1, 0)
    end = np.minimum(end + 1, data.shape[:3])

    slices = [slice(s, e) for s, e in zip(start, end)]

    if return_slices:
        return slices

    return crop_img_to(img, slices, copy=copy)

### Cropping function:

In [7]:
import operator
def cropND(img, bounding):
    start = tuple(map(lambda a, da: a//2-da//2, img.shape, bounding))
    end = tuple(map(operator.add, start, bounding))
    slices = tuple(map(slice, start, end))
    return img[slices]


### Testing crop_img:

In [8]:
# import nibabel as nib
# shapes=[]

In [9]:
# # for patient in completelist:
# #     for extension in ['t1','flair','seg','t2','t1ce']:
# #         img = nib.load("/Users/etheredgej/Desktop/MICCAI_BraTS17_Data_Training/data/"+patient+'_'+extension+".nii.gz")
# #         a = np.array(img.dataobj)
# #         # print(a.shape)
# #         # print(np.unique(a))
# #         # a = np.expand_dims(a, axis=0)
# #         # print(a.shape)
# #         cropped = crop_img(img)
# #         a = np.array(cropped.dataobj)
# #         shapes.append(list(a.shape))
# #         # print(shapes)
# # print(max([shape[0] for shape in shapes])) # 159
# # print(max([shape[1] for shape in shapes])) # 191
# # print(max([shape[2] for shape in shapes])) # 150

# ID = completelist[0]
# img1 = './data/' + ID + '_flair.nii.gz'
# img2 = './data/' + ID + '_t1.nii.gz'
# img3 = './data/' + ID + '_t1ce.nii.gz'
# img4 = './data/' + ID + '_t2.nii.gz'
#             #         img = nib.load("/Users/etheredgej/Desktop/MICCAI_BraTS17_Data_Training/data/"+patient+'_'+extension+".nii.gz")
# #         a = np.array(img.dataobj)
# #         # print(a.shape)
# #         # print(np.unique(a))
# #         # a = np.expand_dims(a, axis=0)
# #         # print(a.shape)
# newimage = nib.concat_images([img1, img2, img3, img4])
# cropped = crop_img(newimage)         
# img_array = np.array(cropped.dataobj)
# b = np.rollaxis(img_array, 3, 0)
# b.shape

# result = np.zeros((4,160,192,160))
# result[:b.shape[0],:b.shape[1],:b.shape[2],:b.shape[3]] = b

# result
# max(set(np.unique(result)))
# # print(result[2][50][50])

# ID = completelist[0]
# imgpath = './data/' + ID + '_flair.nii.gz'
# img = nib.load(imgpath)
# cropped = crop_img(img)
# img_array = np.array(cropped.dataobj)
# b = np.expand_dims(img_array, axis=0)
# result = np.zeros((1,144,144,144))
# result[:b.shape[0],:b.shape[1],:b.shape[2],:b.shape[3]] = b
# result.shape
# np.unique(result)

ID = completelist[0]
imgpath = './data/' + ID + '_flair.nii.gz'
img = nib.load(imgpath)
img_array = np.array(img.dataobj)
img_array = cropND(img_array, (144,144,144))
b = np.expand_dims(img_array, axis=0)
# result = np.zeros((1,144,144,144))
# result[:b.shape[0],:b.shape[1],:b.shape[2],:b.shape[3]] = b
print(b.shape)
# print(np.unique(b))


(1, 144, 144, 144)


### Data generator:

In [10]:
import numpy as np
import keras
import nibabel as nib

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=1, dim=(144,144,144), n_channels=1,
                 n_classes=4, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, self.n_channels, *self.dim))
        y = np.empty((self.batch_size, 1, *self.dim))

        # Generate data
        # Decode and load the data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
#             X[i,] = np.load('data/' + ID + '.npy')

#             imgpath = '/Users/etheredgej/Desktop/MICCAI_BraTS17_Data_Training/data/' + ID + '_flair.nii.gz'
#             img = nib.load(imgpath)
#             img_array = np.array(img.dataobj)
#             X[i,] = np.expand_dims(img_array, axis=0)

            img1 = './data/' + ID + '_flair.nii.gz'
            img2 = './data/' + ID + '_t1.nii.gz'
            img3 = './data/' + ID + '_t1ce.nii.gz'
            img4 = './data/' + ID + '_t2.nii.gz'
            #         img = nib.load("/Users/etheredgej/Desktop/MICCAI_BraTS17_Data_Training/data/"+patient+'_'+extension+".nii.gz")
#         a = np.array(img.dataobj)
#         # print(a.shape)
#         # print(np.unique(a))
#         # a = np.expand_dims(a, axis=0)
#         # print(a.shape)
#         cropped = crop_img(img)
#         a = np.array(cropped.dataobj)
#         shapes.append(list(a.shape))
#         # print(shapes)
#             newimage = nib.concat_images([img1, img2, img3, img4])
#             img_array = np.array(newimage.dataobj)
#             X[i,] = np.rollaxis(img_array, 3, 0)

#             # 4 channel with crop
#             newimage = nib.concat_images([img1, img2, img3, img4])
#             cropped = crop_img(newimage)         
#             img_array = np.array(cropped.dataobj)
#             b = np.rollaxis(img_array, 3, 0)

#             result = np.zeros((4,160,192,160))
#             result[:b.shape[0],:b.shape[1],:b.shape[2],:b.shape[3]] = b

#             X[i,] = result
#             X[i,] = np.expand_dims(img_array, axis=0)

            # single channel with crop beyond zeros (crop to center)
            img = nib.load(img1)
            img_array = np.array(img.dataobj)
            img_array = cropND(img_array, (144,144,144))
            X[i,] = np.expand_dims(img_array, axis=0)

            # y
            # crop single channel (segmentation)
            imgpath = './data/' + ID + '_seg.nii.gz'
            img_array = np.array(img.dataobj)
            img_array = cropND(img_array, (144,144,144))
            y[i,] = np.expand_dims(img_array, axis=0)    
            
#         return X, keras.utils.to_categorical(y, num_classes=self.n_classes)
        return X, y

In [ ]:
import numpy as np

from keras.models import Sequential
# from my_classes import DataGenerator

# Parameters
params = {'dim': (144,144,144),
          'batch_size': 1,
          'n_classes': 4,
          'n_channels': 1,
          'shuffle': True}

# # Datasets
# partition = # IDs
# labels = # Labels

# Generators
training_generator = DataGenerator(partition['train'], labels, **params)
validation_generator = DataGenerator(partition['test'], labels, **params)

# # Design model
# model = Sequential()
# [...] # Architecture
# model.compile()

# Train model on dataset
# model.fit_generator(generator=training_generator,
#                     validation_data=validation_generator,
#                     use_multiprocessing=True,
#                     workers=4)

model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                   epochs=5, 
                   nb_worker=4)

# model.fit_generator(
#         generator=training_generator,
#         samples_per_epoch=2,
#         steps_per_epoch=2000,
#         epochs=50,
#         validation_data=validation_generator,
#         nb_worker=1,
#         validation_steps=800)

/home/ubuntu/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:35: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/home/ubuntu/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:35: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(generator=<__main__...., epochs=5, workers=4, validation_data=<__main__....)`


Epoch 1/5
 11/199 [>.............................] - ETA: 37:15 - loss: -1.7352

KeyboardInterrupt: 

In [ ]:
# score = model.evaluate(x_test, y_test, verbose=0)

score = model.evaluate_generator(generator=validation_generator)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])
score

In [ ]:
285*.3

In [ ]:
# score = model.evaluate(x_test, y_test, verbose=0)

prediction = model.predict_generator(generator=validation_generator)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])
prediction

In [ ]:
prediction.shape

In [ ]:
# ! conda install libgcc